# Summary

-----

# Imports

In [1]:
%run imports.ipynb

2016-09-09 15:39:41.039244


In [2]:
print2 = kmtools.df_tools.print2

In [3]:
NOTEBOOK_NAME = 'interface_data_statistics'

# Load data

In [4]:
db_remote = datapkg.MySQLConnection(
    os.environ['DATAPKG_CONNECTION_STRING'], 
    NOTEBOOK_NAME, 
    None, 
    echo=False)

### Load DATA

This includes both calculated (with $\Delta \Delta G$ prediction) and not calculated (without $\Delta \Delta G$ prediction).

In [5]:
with open('load_data/DATA.pkl', 'rb') as ifh:
    DATA = pickle.load(ifh)

FileNotFoundError: [Errno 2] No such file or directory: 'load_data/DATA.pkl'

In [ ]:
for key in DATA:
    if key in ['humsavar', 'clinvar', 'cosmic']:
        print(key)
        DATA[key] = (
            DATA[key][
                (DATA[key]['uniprot_id_1'].isnull() | 
                 ~DATA[key]['uniprot_id_1'].str.contains('-').astype(bool)) &
                (DATA[key]['uniprot_id_2'].isnull() | 
                 ~DATA[key]['uniprot_id_2'].str.contains('-').astype(bool))
            ]
        )

### Load DATA_DF

This only includes calculated mutations (with $\Delta \Delta G$ prediction).

In [ ]:
DATA_DF = pd.read_pickle('interface_load_data/DATA_DF.pkl')
#DATA_DF['ddg_exp'] = DATA_DF['ddg_exp'].astype(float)

In [ ]:
DATA_DF = (
    DATA_DF[
        (DATA_DF['uniprot_id_1'].isnull() |
         ~DATA_DF['uniprot_id_1'].str.contains('-').astype(bool)) &
        (DATA_DF['uniprot_id_2'].isnull() |
         ~DATA_DF['uniprot_id_2'].str.contains('-').astype(bool))
    ]
)

### Load DATA_DF_TT

In [ ]:
DATA_DF = pd.read_pickle('interface_load_data/DATA_DF_TT.pkl')
#DATA_DF['ddg_exp'] = DATA_DF['ddg_exp'].astype(float)

In [ ]:
DATA_DF_TT = (
    DATA_DF_TT[
        (DATA_DF_TT['uniprot_id_1'].isnull() |
         ~DATA_DF_TT['uniprot_id_1'].str.contains('-').astype(bool)) &
        (DATA_DF_TT['uniprot_id_2'].isnull() |
         ~DATA_DF_TT['uniprot_id_2'].str.contains('-').astype(bool))
    ]
)

### Compare

In [ ]:
DATA['cosmic'][['uniprot_id', 'uniprot_mutation']].drop_duplicates().shape
# 26907

In [ ]:
DATA_DF[DATA_DF['dataset'] == 'cosmic'][['uniprot_id', 'uniprot_mutation']].drop_duplicates().shape
# 26907

# DATA

In [ ]:
DATA.keys()

In [ ]:
header_columns = ['kortemme_baker', 'skempi', 'skempi_database']
tail_columns = ['humsavar', 'clinvar', 'cosmic']

columns_all = (
    header_columns + 
    sorted(c for c in DATA.keys() if c not in header_columns and c not in tail_columns) +
    tail_columns
)
assert not set(DATA.keys()) - set(columns_all)

columns_nodiffseqi = (
    header_columns + 
    sorted(c for c in DATA.keys() if not c.endswith('_database') and 
           c not in header_columns and c not in tail_columns) +
    tail_columns
)

In [ ]:
columns_nodiffseqi

## Dataset overlap

### All columns

In [ ]:
counts = {}
df_out = pd.DataFrame(columns=columns_all, index=columns_all, dtype=float)

for dataset_1 in columns_all:
    # print(dataset_1)
    df_1 = DATA[dataset_1].copy()
    # df_1 = df[df['dataset'] == dataset_1]
    mutation_set_1 = set(df_1['uniprot_id'].astype(str) + '.' + df_1['uniprot_mutation'].astype(str))
    counts[dataset_1] = len(set(
        df_1['pdb_id'].astype(str) + '.' + 
        df_1['uniprot_id'].astype(str) + '.' + 
        df_1['uniprot_mutation'].astype(str)))
    for dataset_2 in columns_all:
        # print('\t', dataset_2)
        df_2 = DATA[dataset_2].copy()
        # df_2 = df[df['dataset'] == dataset_2]
        mutation_set_2 = set(df_2['uniprot_id'].astype(str) + '.' + df_2['uniprot_mutation'].astype(str))
        frac_covered = 1 - len(mutation_set_1 - mutation_set_2) / len(mutation_set_1)
        df_out.loc[dataset_1, dataset_2] = frac_covered * 100.0
df_out.index = ['{}\n(n = {:.0f})'.format(x, counts[x]) for x in df_out.index]

In [ ]:
fg, ax = plt.subplots(figsize=(24, 20))
sns.heatmap(df_out, annot=True, fmt=".2f", ax=ax, square=True, cbar=False, annot_kws={"size": 18})

## Correlations

In [ ]:
# Some validation to make sure that we 
fg, ax = plt.subplots()
ascommon.plotting_tools.make_plot_with_corr(
    x='ddg_exp', 
    y='dg_change',
    data=(
        DATA_DF
        [DATA_DF['dataset'] == 'taipale_ppi']
        # .drop_duplicates(subset=['uniprot_id', 'partner_uniprot_id', 'uniprot_mutation'])
        .groupby(['uniprot_id', 'partner_uniprot_id', 'uniprot_mutation'])
        [['ddg_exp', 'dg_change']]
        # .drop_duplicates()
        .agg('median')
    ),
    ax=ax,
    corr_type='spearman'
)
plt.xlabel('PPI no change (0) vs gain or loss (1)')
plt.ylabel('FoldX $\Delta \Delta G$')

In [ ]:
# Some validation to make sure that we 
fg, ax = plt.subplots()
ascommon.plotting_tools.make_plot_with_corr(
    x='ddg_exp', 
    y='dg_change',
    data=(
        DATA_DF
        [DATA_DF['dataset'] == 'taipale_ppi']
        # .drop_duplicates(subset=['uniprot_id', 'partner_uniprot_id', 'uniprot_mutation'])
        # .groupby(['uniprot_id', 'partner_uniprot_id', 'uniprot_mutation'])
        [['ddg_exp', 'dg_change']]
        # .drop_duplicates()
        # .agg('median')
    ),
    ax=ax,
    corr_type='spearman'
)
plt.xlabel('PPI no change (0) vs gain or loss (1)')
plt.ylabel('FoldX $\Delta \Delta G$')

In [ ]:
# Some validation to make sure that we 
fg, ax = plt.subplots()
ascommon.plotting_tools.make_plot_with_corr(
    x='ddg_exp', 
    y='dg_change',
    data=(
        DATA_DF
        [DATA_DF['dataset'] == 'taipale_gpca']
        # .drop_duplicates(subset=['uniprot_id', 'partner_uniprot_id', 'uniprot_mutation'])
        .groupby(['uniprot_id', 'partner_uniprot_id', 'uniprot_mutation'])
        [['ddg_exp', 'dg_change']]
        # .drop_duplicates()
        .agg('median')
    ), 
    ax=ax,
    corr_type='spearman'
)
plt.xlabel('GPCA score')
plt.ylabel('FoldX $\Delta \Delta G$')

In [ ]:
# Some validation to make sure that we 
fg, ax = plt.subplots()
ascommon.plotting_tools.make_plot_with_corr(
    x='ddg_exp', 
    y='dg_change',
    data=(
        DATA_DF
        [DATA_DF['dataset'] == 'taipale_gpca']
        # .drop_duplicates(subset=['uniprot_id', 'partner_uniprot_id', 'uniprot_mutation'])
        # .groupby(['uniprot_id', 'partner_uniprot_id', 'uniprot_mutation'])
        [['ddg_exp', 'dg_change']]
        # .drop_duplicates()
        # .agg('median')
    ), 
    ax=ax,
    corr_type='spearman'
)
plt.xlabel('GPCA score')
plt.ylabel('FoldX $\Delta \Delta G$')

### Correlation with Foldx

In [ ]:
corrs = OrderedDict()
for table_name in columns_nodiffseqi:
    df = DATA[table_name].drop_duplicates()
    if table_name in ['humsavar_train', 'clinvar_train', 'cosmic_train']:
        df['ddg_exp'] = df['del_class_exp']
    df = df.dropna(subset=['dg_change', 'ddg_exp'])
    corrs[table_name] = sp.stats.spearmanr(df['dg_change'], df['ddg_exp'])[0]
corrs = pd.Series(corrs)

fg, ax = plt.subplots()
x = sns.barplot(corrs.values, corrs.index, ax=ax, color=sns.xkcd_rgb["denim blue"])
plt.xlim(0, 0.6)
ax.set_xlabel('Spearman correlation between experiment\nand FoldX $\Delta \Delta G$')

### Correlation with Provean

In [ ]:
corrs = OrderedDict()
for table_name in columns_nodiffseqi:
    df = DATA[table_name].drop_duplicates()
    if table_name in ['humsavar', 'clinvar', 'cosmic']:
        df['ddg_exp'] = df['del_class_exp']
    else:
        df['ddg_exp'] = df['ddg_exp'].abs()
    df = df.dropna(subset=['provean_score', 'ddg_exp'])
    corrs[table_name] = -sp.stats.spearmanr(df['provean_score'], df['ddg_exp'])[0]
corrs = pd.Series(corrs)

fg, ax = plt.subplots()
x = sns.barplot(corrs.values, corrs.index, ax=ax, color=sns.xkcd_rgb["pale red"])
plt.xlim(0, 0.6)
ax.set_xlabel('Spearman correlation between experiment\nand Provean score (negative)')

# DATA_DF

In [ ]:
DATA_DF.head(2)

In [ ]:
DATA_DF['dataset'].drop_duplicates().tolist()

In [ ]:
datasets = [
    'skempi',
    'taipale_ppi',
    'taipale_gpca',
    'humsavar',
    'clinvar',
    'cosmic',
    'ab_bind',
    'benedix_et_al',
    'hiv_escape_mutations',
]

## Dataset overlap

In [ ]:
columns_training_set = [
    'skempi', 'taipale_ppi', 'taipale_gpca', 'humsavar', 'clinvar', 'cosmic'
]

datasets = [
    # Train
    'skempi',
    # 
    'ab_bind',
    'benedix_et_al',
    'hiv_escape_mutations',
    # Validate
    'taipale_ppi',
    'taipale_gpca',
    # 
    'humsavar_train',
    'clinvar_train',
    'cosmic_train',
    # Test
    'humsavar_test',
    'clinvar_test',
    'cosmic_test',
]

In [ ]:
def get_unique_id(df):
    _df = df[['uniprot_id', 'uniprot_mutation']].dropna()
    uniprot_mutation_set = set(_df['uniprot_id'].astype(str) + '.' + _df['uniprot_mutation'].astype(str))
    
    _df = df[['pdb_id', 'pdb_mutation']].dropna()
    pdb_mutation_set = set(_df['pdb_id'].astype(str) + '.' + _df['pdb_mutation'].astype(str))
    
    return uniprot_mutation_set, pdb_mutation_set

In [ ]:
counts = {}
df = DATA_DF.dropna(subset=['ddg']).copy()
df_out = pd.DataFrame(columns=datasets, index=datasets, dtype=float)

for dataset_1 in datasets:
    # print(dataset_1)
    df_1 = df[df['dataset'] == dataset_1]
    if df_1.empty:
        print(dataset_1)
    uniprot_mutation_set_1, pdb_mutation_set_1 = get_unique_id(df_1)
    counts[dataset_1] = max([len(uniprot_mutation_set_1), len(pdb_mutation_set_1)])
    for dataset_2 in datasets:
        # print('\t', dataset_2)
        df_2 = df[df['dataset'] == dataset_2]
        if df_2.empty:
            print(dataset_2)
        uniprot_mutation_set_2, pdb_mutation_set_2 = get_unique_id(df_2)
        if uniprot_mutation_set_1:
            uniprot_frac_covered = (
                1 - len(uniprot_mutation_set_1 - uniprot_mutation_set_2) / len(uniprot_mutation_set_1))
        else:
            uniprot_frac_covered = 0
        if pdb_mutation_set_1:
            pdb_frac_covered = (
                1 - len(pdb_mutation_set_1 - pdb_mutation_set_2) / len(pdb_mutation_set_1))
        else:
            pdb_frac_covered = 0
        df_out.loc[dataset_1, dataset_2] = max([uniprot_frac_covered, pdb_frac_covered]) * 100.0
df_out.index = ['{}\n(n = {:.0f})'.format(x, counts[x]) for x in df_out.index]

In [ ]:
fg, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(df_out, annot=True, fmt=".1f", ax=ax, square=True, cbar=False, annot_kws={"size": 18})
plt.yticks(rotation='horizontal')
# plt.savefig(op.join(NOTEBOOK_NAME, 'training_set_overlap_final.png'), bbox_inches='tight', dpi=220)
# plt.savefig(op.join(NOTEBOOK_NAME, 'training_set_overlap_final.pdf'), bbox_inches='tight')

In [ ]:
fg, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(df_out, annot=True, fmt=".1f", ax=ax, square=True, cbar=False, annot_kws={"size": 18})
plt.yticks(rotation='horizontal')
plt.savefig(op.join(NOTEBOOK_NAME, 'training_set_overlap_final.png'), bbox_inches='tight', dpi=220)
plt.savefig(op.join(NOTEBOOK_NAME, 'training_set_overlap_final.pdf'), bbox_inches='tight')

## Correlations

### Correlation with FoldX

In [ ]:
corrs = OrderedDict()
for table_name in datasets:
    df = DATA_DF[DATA_DF['dataset'] == table_name].drop_duplicates().copy()
    if any(table_name.startswith(n) for n in ['humsavar', 'clinvar', 'cosmic']):
        df['ddg_exp'] = df['del_class_exp']
        # df['dg_change'] = df['dg_change'].abs()
    df = df.dropna(subset=['dg_change', 'ddg_exp'])
    corrs[table_name] = sp.stats.spearmanr(df['dg_change'], df['ddg_exp'])[0]
corrs = pd.Series(corrs)

fg, ax = plt.subplots()
x = sns.barplot(corrs.values, corrs.index, ax=ax, color=sns.xkcd_rgb["denim blue"])
plt.xlim(0, 0.6)
ax.set_xlabel('Spearman correlation between experiment\nand FoldX $\Delta \Delta G$')
plt.savefig(op.join(NOTEBOOK_NAME, 'foldx_correlation_final.png'), bbox_inches='tight', dpi=220)
plt.savefig(op.join(NOTEBOOK_NAME, 'foldx_correlation_final.pdf'), bbox_inches='tight')

### Correlation with Provean

In [ ]:
corrs = OrderedDict()
for table_name in datasets:
    df = DATA_DF[DATA_DF['dataset'] == table_name].drop_duplicates()
    if any(table_name.startswith(n) for n in ['humsavar', 'clinvar', 'cosmic']):
        df['ddg_exp'] = df['del_class_exp']
    else:
        df['ddg_exp'] = df['ddg_exp'].abs()
        pass
    df = df.dropna(subset=['provean_score', 'ddg_exp'])
    corrs[table_name] = -sp.stats.spearmanr(df['provean_score'], df['ddg_exp'])[0]
corrs = pd.Series(corrs)

fg, ax = plt.subplots()
x = sns.barplot(corrs.values, corrs.index, ax=ax, color=sns.xkcd_rgb["pale red"])
plt.xlim(0, 0.6)
ax.set_xlabel('Spearman correlation between experiment\nand Provean score (negative)')
plt.savefig(op.join(NOTEBOOK_NAME, 'provean_correlation_final.png'), bbox_inches='tight', dpi=220)
plt.savefig(op.join(NOTEBOOK_NAME, 'provean_correlation_final.pdf'), bbox_inches='tight')

In [ ]:
# Compare ELASPIC and FoldX on the test set (NO ABS)

plot_datasets = [
    'skempi', 'taipale_ppi', 'taipale_gpca', 'humsavar', 'clinvar', 'cosmic'
]
plot_dataset_names = {
    'skempi': 'Skempi',
    'taipale_ppi': 'Taipale PPI',
    'taipale_gpca': 'Taipale GPCA' ,
    'humsavar_train': 'Humsavar',
    'clinvar_train': 'Clinvar',
    'cosmic_train': 'Cosmic',
}

fg, axes = plt.subplots(1, 6, figsize=(16, 4))
axes[0].set_ylabel('Spearman R')
for i, test_dataset in enumerate([
        'skempi', 'taipale_ppi', 'taipale_gpca', 
        'humsavar_train', 'clinvar_train', 'cosmic_train']):
    ax = axes[i]
    # print(test_dataset)
    if test_dataset in ['skempi', 'taipale_ppi', 'taipale_gpca']:
        foldx_r = sp.stats.spearmanr(
            DATA_DF[DATA_DF['dataset'] == test_dataset]['dg_change'], 
            DATA_DF[DATA_DF['dataset'] == test_dataset]['ddg_exp'])[0]
        provean_r = sp.stats.spearmanr(
            DATA_DF[DATA_DF['dataset'] == test_dataset]['provean_score'], 
            DATA_DF[DATA_DF['dataset'] == test_dataset]['ddg_exp'])[0] * -1
    else:
        foldx_r = sp.stats.spearmanr(
            DATA_DF[DATA_DF['dataset'] == test_dataset]['dg_change'],  # NO ABS!!!
            DATA_DF[DATA_DF['dataset'] == test_dataset]['del_class_exp'])[0]
        provean_r = sp.stats.spearmanr(
            DATA_DF[DATA_DF['dataset'] == test_dataset]['provean_score'], 
            DATA_DFTT[DATA_DF['dataset'] == test_dataset]['del_class_exp'])[0] * -1
    ax.bar(
        [1, 2], 
        [foldx_r, provean_r],
        tick_label=['FoldX', 'Provean'], 
        align='center', 
        color=[sns.palettes.color_palette()[0], 
               sns.palettes.color_palette()[4]])
    ax.set_ylim(0, 0.60)
    ax.set_title(plot_dataset_names[test_dataset])
    ax.set_xticklabels(labels=ax.get_xticklabels(), rotation=45)
    # print(elaspic_r, foldx_r)

plt.tight_layout()

plt.savefig(op.join(NOTEBOOK_NAME, 'initial_performance.png'), bbox_inches='tight', dpi=220)
plt.savefig(op.join(NOTEBOOK_NAME, 'initial_performance.pdf'), bbox_inches='tight')